In [1]:
# Import all necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import surprise
from surprise.prediction_algorithms import *
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
import numpy as np
import datetime as dt

In [2]:
# Read in the 'links.csv'.
df_links = pd.read_csv("Data/links.csv")
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
# Checking the data.
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [4]:
# Checking the movie ID column for duplicates.
df_links['movieId'].duplicated().sum()

0

# Movies.csv

In [5]:
# Read in the 'movies.csv'.
df_movies = pd.read_csv("Data/movies.csv")
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Checking the data.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [7]:
# Checking the movie ID column for duplicates.
df_movies['movieId'].duplicated().sum()

0

# Rating.csv

In [8]:
# Read in the 'ratings.csv'.
df_ratings = pd.read_csv("Data/ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
# Checking the data
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [10]:
# Checking the movie ID column for duplicates.
df_ratings['movieId'].duplicated().sum()

91112

In [11]:
# Checking the number of user ID's
df_ratings['userId']

0           1
1           1
2           1
3           1
4           1
         ... 
100831    610
100832    610
100833    610
100834    610
100835    610
Name: userId, Length: 100836, dtype: int64

In [12]:
# Double checking user ID's by counting the amount of users.
print(len(df_ratings['userId'].value_counts() > 1))

610


In [13]:
# Looking at the denomination breakdown of ratings.
print((df_ratings['rating'].value_counts()))

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64


# Tags.csv

In [14]:
# Read in 'tags.csv'.
df_tags = pd.read_csv("Data/tags.csv")
df_tags.head()


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [15]:
# Checking the tags data.
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [16]:
# Checking the movie ID column for duplicates.
df_tags['movieId'].duplicated().sum()

2111

In [17]:
# Checking how many moveId's have a value count equal to 1.
df_tagid = df_tags['movieId']
df_tagid = df_tagid.to_frame()
print(len(df_tagid['movieId'].value_counts()==1))


1572


In [18]:
# Counting the number of userId's
print(len(df_tags['userId'].value_counts() > 1))

58


# Merging Dataframes

In [19]:
# Merge 'links' and 'movie' dataframes together and check new dataframe.
movies_df = df_links.merge(df_movies, on='movieId')
movies_df = movies_df.dropna()
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9734 entries, 0 to 9741
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9734 non-null   int64  
 1   imdbId   9734 non-null   int64  
 2   tmdbId   9734 non-null   float64
 3   title    9734 non-null   object 
 4   genres   9734 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 456.3+ KB


In [20]:
# Check to see if there are any movieId duplicates.
movies_df['movieId'].duplicated().sum()

0

In [21]:
# Merge new dataframe with the 'ratings.csv'
movie_df = movies_df.merge(df_ratings, on='movieId')
movie_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100823 entries, 0 to 100822
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100823 non-null  int64  
 1   imdbId     100823 non-null  int64  
 2   tmdbId     100823 non-null  float64
 3   title      100823 non-null  object 
 4   genres     100823 non-null  object 
 5   userId     100823 non-null  int64  
 6   rating     100823 non-null  float64
 7   timestamp  100823 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 6.9+ MB


In [22]:
# Checking to see how many movieIds are duplicated
movie_df['movieId'].duplicated().sum()


91107

In [23]:
print("In this merged dataset we have", len(movie_df['movieId']) - movie_df['movieId'].duplicated().sum(), "individual movie titles.")

In this merged dataset we have 9716 individual movie titles.


In [24]:
movie_df.describe()

,movieId,imdbId,tmdbId,userId,rating,timestamp
count,100823.000000,1.008230e+05,100823.000000,100823.000000,100823.000000,1.008230e+05
mean,19435.437737,3.515704e+05,20105.462633,326.130823,3.501637,1.205945e+09
std,35532.291269,6.220922e+05,53274.143620,182.618176,1.042430,2.162613e+08
min,1.000000,4.170000e+02,2.000000,1.000000,0.500000,8.281246e+08
25%,1199.000000,9.968500e+04,712.000000,177.000000,3.000000,1.019124e+09
50%,2991.000000,1.187710e+05,6957.000000,325.000000,3.500000,1.186087e+09
75%,8120.000000,3.149790e+05,11635.000000,477.000000,4.000000,1.435994e+09
max,193609.000000,8.391976e+06,525662.000000,610.000000,5.000000,1.537799e+09


In [25]:
# We are dropping the 'imdbId' and 'tmdbId' columns because they point to information 
# we do not have access to.
df_cleaned = movie_df.drop(columns= ['imdbId', 'tmdbId'], axis=1)


In [26]:
# Checking the new dataframe.
df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100823 entries, 0 to 100822
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100823 non-null  int64  
 1   title      100823 non-null  object 
 2   genres     100823 non-null  object 
 3   userId     100823 non-null  int64  
 4   rating     100823 non-null  float64
 5   timestamp  100823 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [27]:
# Checking the value counts for different genre combinations.
df_cleaned['genres'].value_counts()


Comedy                                     7194
Drama                                      6290
Comedy|Romance                             3967
Comedy|Drama|Romance                       3000
Comedy|Drama                               2851
                                           ... 
Documentary|Fantasy                           1
Action|Horror|Mystery|Sci-Fi                  1
Comedy|Documentary|Romance                    1
Comedy|Drama|Sci-Fi|War                       1
Animation|Children|Comedy|Drama|Romance       1
Name: genres, Length: 951, dtype: int64

# Filtering by rating to recommend movie with a rating of 3.0 and above

In [28]:
# df_clean will only contain ratings 3.0 and greater.
df_clean = df_cleaned[df_cleaned['rating'] >= 3.0]

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81754 entries, 0 to 100822
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    81754 non-null  int64  
 1   title      81754 non-null  object 
 2   genres     81754 non-null  object 
 3   userId     81754 non-null  int64  
 4   rating     81754 non-null  float64
 5   timestamp  81754 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 4.4+ MB


In [29]:
# Movies with the highest average rating into separate df.
avg_high_rated = df_clean.groupby(['title']).agg({"rating":"mean"})['rating'].sort_values(ascending=False)
avg_high_rated = avg_high_rated.to_frame()
avg_high_rated.reset_index(level=0, inplace=True)
avg_high_rated.columns = ['title', 'Average Rating']
avg_high_rated.head(2)


,title,Average Rating
0,"American Friend, The (Amerikanische Freund, De...",5.0
1,"Four Days in September (O Que É Isso, Companhe...",5.0


In [30]:
# Split the genres into separate columns with a binary result.
genres_split = df_clean.genres.apply(lambda x: x.split(sep='|')).apply(pd.value_counts, 1).fillna(0)
df_clean = pd.concat([df_clean.iloc[:,:], genres_split], axis=1)
df_clean.head(2)


,movieId,title,genres,userId,rating,timestamp,Animation,Children,Fantasy,Adventure,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df_clean['(no genres listed)'].sum()

35.0

In [32]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81754 entries, 0 to 100822
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             81754 non-null  int64  
 1   title               81754 non-null  object 
 2   genres              81754 non-null  object 
 3   userId              81754 non-null  int64  
 4   rating              81754 non-null  float64
 5   timestamp           81754 non-null  int64  
 6   Animation           81754 non-null  float64
 7   Children            81754 non-null  float64
 8   Fantasy             81754 non-null  float64
 9   Adventure           81754 non-null  float64
 10  Comedy              81754 non-null  float64
 11  Romance             81754 non-null  float64
 12  Drama               81754 non-null  float64
 13  Crime               81754 non-null  float64
 14  Thriller            81754 non-null  float64
 15  Action              81754 non-null  float64
 16  Hor

In [33]:
# Extract the release year from the 'title' column into a new 'year' column.
import re

def extract_year(title):
    match = re.search('\((\d{4})\)', title)
    if match:
        return match.group(1)
    else:
        return None

df_clean['year'] = df_clean['title'].apply(extract_year)

In [34]:
# Checking for missing year values.
df_clean['year'].isna().sum()


14

In [35]:
# Dropping missing values.
df_clean.dropna(subset=['year'], inplace=True)
df_clean['year'].isna().sum()


0

In [36]:
df_clean['num_viewers'] = df_clean.groupby('movieId')['userId'].transform('count')


In [37]:
df_clean.head(2)

,movieId,title,genres,userId,rating,timestamp,Animation,Children,Fantasy,Adventure,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),year,num_viewers
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1995,199
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1995,199


In [38]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81740 entries, 0 to 100822
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             81740 non-null  int64  
 1   title               81740 non-null  object 
 2   genres              81740 non-null  object 
 3   userId              81740 non-null  int64  
 4   rating              81740 non-null  float64
 5   timestamp           81740 non-null  int64  
 6   Animation           81740 non-null  float64
 7   Children            81740 non-null  float64
 8   Fantasy             81740 non-null  float64
 9   Adventure           81740 non-null  float64
 10  Comedy              81740 non-null  float64
 11  Romance             81740 non-null  float64
 12  Drama               81740 non-null  float64
 13  Crime               81740 non-null  float64
 14  Thriller            81740 non-null  float64
 15  Action              81740 non-null  float64
 16  Hor

In [39]:
# Change year column to integer.
df_clean['year'] = df_clean['year'].astype('int')


In [40]:
# Creating df_clean2 to have copy before dividing the years into decades.
df_clean_col = df_clean.copy()
df_clean2 = df_clean_col.merge(avg_high_rated, on='title')
df_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81740 entries, 0 to 81739
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             81740 non-null  int64  
 1   title               81740 non-null  object 
 2   genres              81740 non-null  object 
 3   userId              81740 non-null  int64  
 4   rating              81740 non-null  float64
 5   timestamp           81740 non-null  int64  
 6   Animation           81740 non-null  float64
 7   Children            81740 non-null  float64
 8   Fantasy             81740 non-null  float64
 9   Adventure           81740 non-null  float64
 10  Comedy              81740 non-null  float64
 11  Romance             81740 non-null  float64
 12  Drama               81740 non-null  float64
 13  Crime               81740 non-null  float64
 14  Thriller            81740 non-null  float64
 15  Action              81740 non-null  float64
 16  Horr

In [41]:
df_clean2 = df_clean2.drop(columns='timestamp', axis=1)

In [42]:
df_clean2 = df_clean2.drop_duplicates(subset='title', keep='first')
df_clean2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8432 entries, 0 to 81739
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             8432 non-null   int64  
 1   title               8432 non-null   object 
 2   genres              8432 non-null   object 
 3   userId              8432 non-null   int64  
 4   rating              8432 non-null   float64
 5   Animation           8432 non-null   float64
 6   Children            8432 non-null   float64
 7   Fantasy             8432 non-null   float64
 8   Adventure           8432 non-null   float64
 9   Comedy              8432 non-null   float64
 10  Romance             8432 non-null   float64
 11  Drama               8432 non-null   float64
 12  Crime               8432 non-null   float64
 13  Thriller            8432 non-null   float64
 14  Action              8432 non-null   float64
 15  Horror              8432 non-null   float64
 16  Myste

In [43]:
df_clean2['genres'] = df_clean2['genres'].str.replace('|', ' ')


In [44]:
# Creating columns for each decade span with a binary value for each movie.
df_clean['1900s'] = ((df_clean['year'] >= 1900) & (df_clean['year'] < 1910)).astype(float)
df_clean['1910s'] = ((df_clean['year'] >= 1910) & (df_clean['year'] < 1920)).astype(float)
df_clean['1920s'] = ((df_clean['year'] >= 1920) & (df_clean['year'] < 1930)).astype(float)
df_clean['1930s'] = ((df_clean['year'] >= 1930) & (df_clean['year'] < 1940)).astype(float)
df_clean['1940s'] = ((df_clean['year'] >= 1940) & (df_clean['year'] < 1950)).astype(float)
df_clean['1950s'] = ((df_clean['year'] >= 1950) & (df_clean['year'] < 1960)).astype(float)
df_clean['1960s'] = ((df_clean['year'] >= 1960) & (df_clean['year'] < 1970)).astype(float)
df_clean['1970s'] = ((df_clean['year'] >= 1970) & (df_clean['year'] < 1980)).astype(float)
df_clean['1980s'] = ((df_clean['year'] >= 1980) & (df_clean['year'] < 1990)).astype(float)
df_clean['1990s'] = ((df_clean['year'] >= 1990) & (df_clean['year'] < 2000)).astype(float)
df_clean['2000s'] = ((df_clean['year'] >= 2000) & (df_clean['year'] < 2010)).astype(float)
df_clean['2010s'] = ((df_clean['year'] >= 2010) & (df_clean['year'] < 2020)).astype(float)


In [45]:
df_clean.head(2)

,movieId,title,genres,userId,rating,timestamp,Animation,Children,Fantasy,Adventure,...,1920s,1930s,1940s,1950s,1960s,1970s,1980s,1990s,2000s,2010s
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [46]:
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 81740 entries, 0 to 100822
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             81740 non-null  int64  
 1   title               81740 non-null  object 
 2   genres              81740 non-null  object 
 3   userId              81740 non-null  int64  
 4   rating              81740 non-null  float64
 5   timestamp           81740 non-null  int64  
 6   Animation           81740 non-null  float64
 7   Children            81740 non-null  float64
 8   Fantasy             81740 non-null  float64
 9   Adventure           81740 non-null  float64
 10  Comedy              81740 non-null  float64
 11  Romance             81740 non-null  float64
 12  Drama               81740 non-null  float64
 13  Crime               81740 non-null  float64
 14  Thriller            81740 non-null  float64
 15  Action              81740 non-null  float64
 16  Hor

In [47]:
#Dropping these two columns as they are not needed for further analysis
df_clean.drop(columns=['(no genres listed)','timestamp'],inplace=True)
